In [1]:
import osmium 
import pandas as pd
import geopandas as gpd
import glob
import os

[GeoFabric Russian Federal Districts](https://download.geofabrik.de/russia.html)

In [2]:
# Папка с дампами федеральных округов России из ссылки выше
input_diles = glob.glob('/Users/maximgrinin/Downloads/*.osm.pbf')

In [3]:
# Список OSM тэгов
tag_list = [
    {'station': ['subway']},
    {'highway': ['stop_position']},
    {'government': ['administrative']},
    {'amenity': ['college', 'university']},
    {'amenity': ['school']},
    {'amenity': ['kindergarten', 'childcare']},
    {'amenity': ['bank']},
    {'amenity': ['atm']},
    {'amenity': ['hospital']},
    {'office': ['company']},
    {'amenity': ['cafe', 'fast_food', 'bar', 'pub', 'canteen']},
    {'government': ['public_service']},
    {'building': ['apartments']},
    {'aeroway': ['aerodrome']},
    {'railway': ['station']}
]

# Human-readable имена тегов 
tag_names = [
    'subway',
    'bus_stop',
    'gov_building',
    'college',
    'school',
    'kgarden',
    'bank',
    'atm',
    'hospital',
    'office',
    'food',
    'MFC',
    'apartments',
    'airport',
    'railway'
]

tags = dict(zip(tag_names, tag_list))

In [4]:
def get_osmium_script(initial_file, output_file, tags):
    
    '''Функция возвращает скрипт osmium-tool обрезающую pbf по листу тегов
    
    initial_file -- путь к целому pbf файлу
    output_file --  название/путь файла обрезанного pbf файла 
    tags -- словарь с osm тегами
    '''
    
    osmium_string = f'osmium tags-filter {initial_file} '
    for name, tag in tags.items():
        for key, value_list in tag.items():
            if len(value_list) == 1:
                osmium_string += r'n/{}={} '.format(key, value_list[0])
            else:
                for value in value_list:
                    osmium_string += r'n/{}={} '.format(key, value)
    osmium_string += f'-o {output_file}'
    
    return osmium_string

In [5]:
# Собираем лист с путями к цельным pbf файлам
input_files = glob.glob('/Users/maximgrinin/Downloads/full_pbf/*.osm.pbf')

# Собираем скрипт для каждого файла из списка
# Исполняем их и сохраняем обрезанные файлы в отдельную дерикторию fed_districts
for file in input_files:
    file_name = file.replace('/Users/maximgrinin/Downloads/full_pbf/', '')
    os.system(get_osmium_script(file, 'fed_districts/cutted-{}'.format(file_name), tags))

In [6]:
# Создаем SimpleHandler для парсинга нужной информации из обрезанных pbf
class OSMHandler(osmium.SimpleHandler):
    def __init__(self):
        super(OSMHandler, self).__init__()
        self.infrastructure = []

    def node(self, o):
        for name, tag in tags.items():
            for key, value_list in tag.items():
                if len(value_list) == 1:
                    if o.tags.get(key) == value_list[0]:
                        self.infrastructure.append(
                            [name, o.location.lon, o.location.lat]
                        )
                else:
                    for value in value_list:
                        if o.tags.get(key) == value:
                            self.infrastructure.append(
                                [name, o.location.lon, o.location.lat]
                            )

In [10]:
# Названия обрезанных pbf файлов
osm_files = glob.glob('fed_districts/*.osm.pbf')

# Инициализируем словарик геофреймов
districts_gdfs = {}

# Итерируем над обрезанными pbf файлами
# Парсим из них инфо по тегам
# Записываем в геофрейм и крепим к словарю
for district in osm_files:
    handler = OSMHandler()
    handler.apply_file(district, locations=True)
    df = pd.DataFrame(handler.infrastructure, columns=['type', 'lon', 'lat'])
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']))
    gdf.crs = 'EPSG:4326'
    districts_gdfs[district.replace('fed_districts/', '').replace('-latest.osm.pbf', '')] = gdf

In [13]:
# Сохраняем все в csv
for name, gdf in districts_gdfs.items():
    gdf.to_csv('fed_districts/' + name + '.csv')